In [2]:
import os
import random
import shutil
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageOps
import numpy as np

import numpy as np
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
from tqdm import tqdm
from torchvision.utils import save_image
from PIL import Image

import os
import cv2
import numpy as np
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageFilter

import random


OUTPUT_BASE = "data_augmented"
os.makedirs(OUTPUT_BASE, exist_ok=True)
TRAIN_DATA_DIR = "train/"

# --- CONFIG ---
IMG_SIZE = (150, 150)
OUTPUT_BASE = "data_augmented"
SINGLE_DIR = os.path.join(OUTPUT_BASE, "single_various_augmentation")
MULTI_DIR = os.path.join(OUTPUT_BASE, "multiple_various_augmentation")

# make sure base directories exist
os.makedirs(SINGLE_DIR, exist_ok=True)
os.makedirs(MULTI_DIR, exist_ok=True)

# --- Moderate augmentation parameters ---
MODERATE_PARAMS = {
    "rotation": (-30, 30),              # degrees
    "translation": (-0.2, 0.2),         # fraction of image size
    "scaling": (0.9, 1.1),              # zoom in/out
    "shear": (-15, 15),                 # degrees
    "h_flip": 0.5,                       # probability
    "v_flip": 0.3,                       # probability
    "brightness": (0.8, 1.2),            # only RGB
    "contrast": (0.8, 1.2),              # only RGB
    "noise_std": (0.01, 0.05),           # Gaussian noise
    "blur_radius": (0, 1)                # Gaussian blur
}

In [ ]:
# === Utility Functions ===

def ensure_dir(path, recreate=False):
    if recreate and os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

def add_gaussian_noise(img, sigma=0.05):
    np_img = np.array(img).astype(np.float32) / 255.0
    noise = np.random.normal(0, sigma, np_img.shape)
    np_img = np.clip(np_img + noise, 0, 1)
    return Image.fromarray((np_img * 255).astype(np.uint8))

def random_erasing(img, erase_prob=0.3):
    if random.random() > erase_prob:
        return img
    np_img = np.array(img)
    h, w, _ = np_img.shape
    erase_w = random.randint(int(w * 0.05), int(w * 0.2))
    erase_h = random.randint(int(h * 0.05), int(h * 0.2))
    x1 = random.randint(0, w - erase_w)
    y1 = random.randint(0, h - erase_h)
    np_img[y1:y1+erase_h, x1:x1+erase_w, :] = np.random.randint(0, 255, (erase_h, erase_w, 3), dtype=np.uint8)
    return Image.fromarray(np_img)

def apply_affine(img, rotation=0, tx=0, ty=0, shear=0, scale=1.0):
    w, h = img.size
    xshift = tx * w
    yshift = ty * h
    return img.transform(
        img.size,
        Image.AFFINE,
        (scale * np.cos(np.radians(shear)), -np.sin(np.radians(shear)), xshift,
         np.sin(np.radians(shear)), scale * np.cos(np.radians(shear)), yshift),
        resample=Image.BICUBIC
    )

# === Augmentation Modes ===

def augment_geo_flip_heavy(img):
    if random.random() < 0.8:
        img = ImageOps.mirror(img)
    if random.random() < 0.6:
        img = ImageOps.flip(img)
    if random.random() < 0.8:
        angle = random.uniform(-10, 10)
        img = img.rotate(angle)
    if random.random() < 0.6:
        tx, ty = random.uniform(-0.05, 0.05), random.uniform(-0.05, 0.05)
        img = apply_affine(img, tx=tx, ty=ty)
    if random.random() < 0.5:
        scale = random.uniform(0.9, 1.1)
        img = apply_affine(img, scale=scale)
    if random.random() < 0.5:
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(random.uniform(0.85, 1.15))
    if random.random() < 0.5:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(random.uniform(0.85, 1.15))
    return img


def augment_cut_noise_heavy(img):
    if random.random() < 0.6:
        angle = random.uniform(-8, 8)
        img = img.rotate(angle)
    if random.random() < 0.7:
        tx, ty = random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)
        img = apply_affine(img, tx=tx, ty=ty)
    if random.random() < 0.6:
        img = add_gaussian_noise(img, sigma=0.05)
    if random.random() < 0.7:
        img = random_erasing(img, erase_prob=0.4)
    if random.random() < 0.6:
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(random.uniform(0.75, 1.25))
    if random.random() < 0.6:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(random.uniform(0.7, 1.3))
    return img


def augment_vertical_stretch_heavy(img):
    if random.random() < 0.7:
        angle = random.uniform(-20, 20)
        img = img.rotate(angle)
    if random.random() < 0.7:
        scale_y = random.uniform(0.8, 1.2)
        w, h = img.size
        img = img.resize((w, int(h * scale_y)))
    if random.random() < 0.6:
        shear = random.uniform(-15, 15)
        img = apply_affine(img, shear=shear)
    if random.random() < 0.5:
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(random.uniform(0.8, 1.2))
    if random.random() < 0.5:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(random.uniform(0.8, 1.2))
    return img


# === Main Function ===

def augment_and_aggregate(TRAIN_DATA_DIR, OUTPUT_BASE="data_augmented", img_size=(150, 150), variants_per_image=8):
    modes = {
        "geo_flip_heavy": augment_geo_flip_heavy,
        #"cut_noise_heavy": augment_cut_noise_heavy,
        #"vertical_stretch_heavy": augment_vertical_stretch_heavy,
    }

    # Step 1: Create individual datasets
    print(f"🧩 Creating augmentation datasets in '{OUTPUT_BASE}'...")
    ensure_dir(OUTPUT_BASE, recreate=True)
    for mode in modes:
        ensure_dir(os.path.join(OUTPUT_BASE, mode), recreate=True)

    class_names = sorted(os.listdir(TRAIN_DATA_DIR))
    for class_name in class_names:
        class_path = os.path.join(TRAIN_DATA_DIR, class_name)
        if not os.path.isdir(class_path):
            continue

        print(f"\nProcessing class: {class_name}")
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Create subfolders per class per mode
        for mode in modes:
            ensure_dir(os.path.join(OUTPUT_BASE, mode, class_name))

        for img_name in tqdm(image_files, desc=f"{class_name}", ncols=80):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path).convert("RGB").resize(img_size)

            for mode_name, func in modes.items():
                for i in range(variants_per_image):
                    aug_img = func(img.copy())
                    out_name = f"{os.path.splitext(img_name)[0]}_{mode_name}_aug{i}.jpg"
                    out_path = os.path.join(OUTPUT_BASE, mode_name, class_name, out_name)
                    aug_img.save(out_path, "JPEG", quality=90)

    # Step 2: Aggregate all augmentations into one combined dataset
    print("\n🧱 Aggregating all augmentations into one dataset...")
    aggregate_dir = os.path.join(OUTPUT_BASE, "aggregate")
    ensure_dir(aggregate_dir, recreate=True)

    for class_name in class_names:
        ensure_dir(os.path.join(aggregate_dir, class_name))
        for mode_name in modes.keys():
            src_dir = os.path.join(OUTPUT_BASE, mode_name, class_name)
            if not os.path.exists(src_dir):
                continue
            for img_file in os.listdir(src_dir):
                src = os.path.join(src_dir, img_file)
                dst = os.path.join(aggregate_dir, class_name, img_file)
                shutil.copy2(src, dst)

    print("\n✅ All augmentation datasets and aggregate dataset created successfully!")
    print(f"📁 Aggregate dataset saved to: {aggregate_dir}")

: 

In [3]:
augment_and_aggregate(
    TRAIN_DATA_DIR, 
    OUTPUT_BASE=OUTPUT_BASE,
    img_size=(150, 150),
    variants_per_image=4
)


🧩 Creating augmentation datasets in 'data_augmented'...

Processing class: apple


apple: 100%|█████████████████████████████████| 230/230 [00:01<00:00, 159.70it/s]



Processing class: avocado


avocado: 100%|███████████████████████████████| 228/228 [00:01<00:00, 152.22it/s]



Processing class: banana


...


Processing class: strawberries


strawberries: 100%|██████████████████████████| 230/230 [00:01<00:00, 161.58it/s]



Processing class: watermelon


watermelon: 100%|████████████████████████████| 229/229 [00:01<00:00, 166.52it/s]



🧱 Aggregating all augmentations into one dataset...

✅ All augmentation datasets and aggregate dataset created successfully!
📁 Aggregate dataset saved to: data_augmented/aggregate


In [ ]:
# --- CONFIG ---
BATCH_SIZE = 500
EPOCHS = 100
NUM_CLASSES = 10
DATA_AUG_DIR = OUTPUT_BASE
VAL_DATA_DIR = "data/val"

In [9]:
# --- CONFIG ---
BATCH_SIZE = 500
EPOCHS = 100
NUM_CLASSES = 10
DATA_AUG_DIR = OUTPUT_BASE
VAL_DATA_DIR = "data/val"

OUTPUT_DIR = os.path.join(DATA_AUG_DIR, "geo_flip_heavy")

# --- Load training dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    OUTPUT_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# --- Resize validation dataset to match training ---
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Define CNN model ---
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- Callbacks ---
checkpoint_path = os.path.join(f"best_model_{OUTPUT_DIR}.keras")
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# --- Train ---
history_geo = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
    verbose=2
)

# --- Store best validation accuracy ---
best_val_loss = max(history_geo.history['val_loss'])

Found 18352 files belonging to 10 classes.
Found 1025 files belonging to 10 classes.
Epoch 1/100

Epoch 1: val_loss improved from inf to 2.26151, saving model to best_model_data_augmented/geo_flip_heavy.keras
37/37 - 20s - 536ms/step - accuracy: 0.1067 - loss: 2.2931 - val_accuracy: 0.1102 - val_loss: 2.2615
Epoch 2/100

Epoch 2: val_loss improved from 2.26151 to 1.92391, saving model to best_model_data_augmented/geo_flip_heavy.keras
37/37 - 20s - 529ms/step - accuracy: 0.2149 - loss: 2.0740 - val_accuracy: 0.2693 - val_loss: 1.9239
Epoch 3/100

Epoch 3: val_loss improved from 1.92391 to 1.76497, saving model to best_model_data_augmented/geo_flip_heavy.keras
37/37 - 21s - 557ms/step - accuracy: 0.3324 - loss: 1.7038 - val_accuracy: 0.3024 - val_loss: 1.7650
Epoch 4/100

Epoch 4: val_loss improved from 1.76497 to 1.69323, saving model to best_model_data_augmented/geo_flip_heavy.keras
37/37 - 21s - 568ms/step - accuracy: 0.3682 - loss: 1.6019 - val_accuracy: 0.3561 - val_loss: 1.6932
Epo

In [ ]:
# --- CONFIG ---
BATCH_SIZE = 500
EPOCHS = 100
NUM_CLASSES = 10
DATA_AUG_DIR = OUTPUT_BASE
VAL_DATA_DIR = "data/val"

OUTPUT_DIR = os.path.join(DATA_AUG_DIR, "cut_noise_heavy_augmentation")

# --- Load training dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    OUTPUT_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# --- Resize validation dataset to match training ---
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Define CNN model ---
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- Callbacks ---
checkpoint_path = os.path.join(f"best_model_{OUTPUT_DIR}.keras")
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# --- Train ---
history_cut = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
    verbose=2
)

# --- Store best validation accuracy ---
best_val_loss = max(history_cut.history['val_loss'])

In [ ]:
# --- CONFIG ---
BATCH_SIZE = 500
EPOCHS = 100
NUM_CLASSES = 10
DATA_AUG_DIR = OUTPUT_BASE
VAL_DATA_DIR = "data/val"

OUTPUT_DIR = os.path.join(DATA_AUG_DIR, "vert_stretch_heavy_augmentation")

# --- Load training dataset ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    OUTPUT_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# --- Resize validation dataset to match training ---
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Define CNN model ---
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(5, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- Callbacks ---
checkpoint_path = os.path.join(f"best_model_{OUTPUT_DIR}.keras")
checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# --- Train ---
history_vert = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb],
    verbose=2
)

# --- Store best validation accuracy ---
best_val_loss = max(history_vert.history['val_loss'])